# Proyecto 2
Todas las preguntas tienen el mismo peso en esta tarea. Esta tarea requiere más aprendizaje individual. Te invito a consultar la [documentación de pandas](https://pandas.pydata.org/pandas-docs/stable/) para encontrar funciones o métodos que quizás no has usado aún, o realices preguntas en [Stack Overflow](https://stackoverflow.com/) y etiquételos como relacionados con pandas y python. Todas las preguntas tienen la misma cantidad de puntos, excepto la pregunta 1, que vale el 20 % de la calificación de la tarea.

**Nota**: A partir de la pregunta 2 se basan en la respuesta de la pregunta 1.

In [7]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings

warnings.filterwarnings('ignore')

### Pregunta 1
Cargue los datos de energía del archivo `assets/Energy Indicators.xls`, que es una lista de indicadores de [suministro de energía y producción de electricidad renovable](assets/Energy%20Indicators.xls) de las [Naciones Unidas](https://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) para el año 2013, y asigna esos datos a un DataFrame con el nombre de variable **energy**.

Ten en cuenta que este es un archivo de Excel y no un archivo de valores separados por comas (.csv). Además, asegúrate de excluir la información del pie de página y del encabezado del archivo de datos. Las primeras dos columnas son innecesarias, por lo que debes deshacerse de ellas. También debes cambiar las etiquetas de las columnas para que las columnas sean las siguientes:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convierta `Energy Supply` a gigajulios (**Nota: hay 1,000,000 giga julios en un petajulio**). Para todos los países que carezcan de datos (por ejemplo, datos con "..."), asegúrese de que estos reflejen valores `np.NaN`.

Cambia el nombre de los siguientes países según este listado (nos servirá para preguntas posteriores):

```
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

También hay varios países que contienen paréntesis en su nombre. Asegúrete de eliminarlos, por ejemplo: `'Bolivia (Estado Plurinacional de)'` debería ser `'Bolivia'`.

A continuación, carga los datos del GDP (PBI) del archivo `assets/world_bank.csv`. Este es un archivo .csv del [Banco Mundial](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD) que contiene el PBI de los países desde 1960 hasta 2015. Llame a este DataFrame **gdp**.

Asegúrete de omitir el encabezado y cambia el nombre de la siguiente lista de países:

```
"Korea, Rep.": "South Korea",
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finalmente, lee el [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](https://www.scimagojr.com/countryrank.php?category=2102) desde el archivo `assets/scimagojr-3.xlsx`, que clasifica a los paises según sus contribuciones al área mencionada. Denomina este DataFrame con el nombre **sciamgo**.

Unifica los tres datasets: GDP, Energy, y ScimEn en un nuevo dataset (utilizando la intersección de los nombres de países). Utiliza solo los años del 2006 al 2015 de datos de GDP (PBI) y solo los 15 principales países según el ranking de Scimagojr 'Rank' (del 1 al 15).

El índice de este DataFrame debe ser el nombre de los países, y las columnas deben ser ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

*Esto debería devolver un DataFrame con 20 columnas y 15 entradas, y las filas del DataFrame deberían estar ordenadas por "Rank".*


In [29]:
import pandas as pd
import numpy as np

# Cargar los datos de Energy Indicators.xls
energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38, usecols=[2, 3, 4, 5], names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x * 1_000_000 if not pd.isna(x) else x)
energy['Country'] = energy['Country'].str.replace(r'\d+', '')  # Eliminar dígitos del nombre de los países
energy['Country'] = energy['Country'].str.replace(r'\(.*\)', '')  # Eliminar texto entre paréntesis
energy['Country'] = energy['Country'].str.strip()  # Eliminar espacios en blanco al principio y al final

# Cambiar nombres de países
country_name_changes = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
energy['Country'].replace(country_name_changes, inplace=True)

# Cargar los datos del GDP (PBI) desde world_bank.csv
gdp = pd.read_csv('assets/world_bank.csv', skiprows=4)
gdp['Country Name'] = gdp['Country Name'].replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
})

# Cargar el Scimago Journal and Country Rank data
scimago = pd.read_excel('assets/scimagojr-3.xlsx')

# Seleccionar los años del 2006 al 2015 del GDP (PBI)
gdp = gdp[['Country Name'] + [str(i) for i in range(2006, 2016)]]

# Seleccionar los 15 países principales según el ranking de Scimagojr
scimago = scimago[scimago['Rank'] <= 15]

# Unir los tres conjuntos de datos en un nuevo DataFrame
merged_df = pd.merge(scimago, energy, left_on='Country', right_on='Country', how='inner')
merged_df = pd.merge(merged_df, gdp, left_on='Country', right_on='Country Name', how='inner')

# Establecer el índice en el nombre del país y seleccionar las columnas requeridas
merged_df = merged_df.set_index('Country')[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + [str(i) for i in range(2006, 2016)]]

# Ordenar por "Rank"
merged_df = merged_df.sort_values(by='Rank')

# Verificar el DataFrame resultante
print(merged_df)


                    Rank  Documents  Citable documents  Citations  \
Country                                                             
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
South Korea           10      11983              11923     114675   
Brazil                15       8668               8596      60702   

                    Self-citations  Citations per document  H index  \
Country                                                               
Russian Federation           12422                    1.85       57   
Canada                       40930                   12.01      149   
Germany                      27426                    8.26      126   
India                        37209                    8.58      115   
South Korea          

### Pregunta 2
La pregunta anterior unió tres datasets y luego los redujo a las 15 entradas principales. Pero antes de reducirlos a los 15 elementos principales, ¿cuántas entradas se perdieron?

*Esta función debería devolver un solo número.*

In [41]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Todo menos esto!</text>
</svg>

In [51]:
total_entries_before = len(scimago)  # Número de entradas en el conjunto de datos de Scimago
total_entries_after = len(merged_df)  # Número de entradas después de la reducción a las 15 principales

entradas_perdidas = total_entries_before - total_entries_after

entradas_perdidas


9

### Pregunta 3
¿Cuáles son los 15 países con mayor GDP (PBI) promedio en los últimos 10 años?

*Esta función debería devolver una serie llamada `avgGDP` con 15 países y su GDP (PBI) promedio ordenado en orden descendente.*

In [285]:
# Calcular el PBI promedio para cada país en el período de 2006 a 2015
gdp_columns = [str(i) for i in range(2006, 2016)]
gdp['Avg GDP'] = gdp[gdp_columns].mean(axis=1)

# Filtrar las filas que corresponden a las entradas que no son países individuales
non_country_entries = ['World', 'High income', 'OECD members', 'Post-demographic dividend', 'IDA & IBRD total', 'Europe & Central Asia', 'Low & middle income', 'Middle income', 'IBRD only', 'European Union', 'East Asia & Pacific', 'North America', 'Upper middle income', 'Late-demographic dividend', 'Euro area', 'Early-demographic dividend', 'East Asia & Pacific (excluding high income)', 'East Asia & Pacific (IDA & IBRD countries)', 'Latin America & Caribbean', 'Latin America & the Caribbean (IDA & IBRD countries)', 'Lower middle income', 'Latin America & Caribbean (excluding high income)', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)', 'Middle East & North Africa', 'South Asia', "South Asia (IDA & IBRD)", "Arab World", "IDA total", "Sub-Saharan Africa", 'Sub-Saharan Africa (IDA & IBRD countries)', 'Sub-Saharan Africa (excluding high income)', 'Central Europe and the Baltics', "Middle East & North Africa (excluding high income)", "Middle East & North Africa (IDA & IBRD countries)"]

avgGDP = gdp[~gdp['Country'].isin(non_country_entries)]

# Ordenar los países en orden descendente según el PBI promedio
avgGDP = avgGDP[['Country', 'Avg GDP']].sort_values(by='Avg GDP', ascending=False)

# Seleccionar los 15 primeros países
top15_avgGDP = avgGDP.head(15)

# Establecer 'Country Name' como índice y eliminar la columna
top15_avgGDP = top15_avgGDP.set_index('Country')['Avg GDP']

top15_avgGDP

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Mexico                1.090990e+12
Name: Avg GDP, dtype: float64

### Pregunta 4
¿Cuánto cambió el GDP (PBI) durante el lapso de 10 años para el país con el sexto GDP (PBI) promedio más grande?

*Esta función debería devolver un solo número.*

In [293]:
# Obtener el PBI en 2006 y 2015 para el Reino Unido (6to PBI más alto)
gdp_uk_2006 = gdp.loc[gdp['Country'] == 'United Kingdom', '2006'].values[0]
gdp_uk_2015 = gdp.loc[gdp['Country'] == 'United Kingdom', '2015'].values[0]

# Calcular el cambio en el PBI durante 10 años para el Reino Unido
gdp_change_uk = gdp_uk_2015 - gdp_uk_2006

gdp_change_uk

246702696075.3999

### Pregunta 5
¿Cuál es el suministro medio de energía per cápita (`Energy Supply per Capita`)?

*Esta función debería devolver un solo número.*

In [301]:
mean_energy_supply_per_capita = merged_df['Energy Supply per Capita'].mean()
print(mean_energy_supply_per_capita)


163.5


### Pregunta 6
¿Qué país tiene el máximo `% Renewable` y cuál es el porcentaje?

*Esta función debe devolver una tupla con el nombre del país y el porcentaje.*

In [312]:
# Encontrar el país con el máximo % Renewable
max_renewable_country = merged_df['% Renewable'].idxmax()  # Obtener el índice del país con el máximo % Renewable
max_renewable_percentage = merged_df['% Renewable'].max()  # Obtener el valor máximo de % Renewable

# Crear una tupla con el nombre del país y el porcentaje máximo de % Renewable
result = (max_renewable_country, max_renewable_percentage)

# Imprimir el resultado
print(result)


('Brazil', 69.64803)


### Pregunta 7
Cree una nueva columna que sea la relación entre `Self-Citations` y `Total Citations`.
¿Cuál es el valor máximo para esta nueva columna y qué país tiene la relación más alta?

*Esta función debe devolver una tupla con el nombre del país y el ratio (proporción).*

In [317]:
# Crear la nueva columna "Citations Ratio"
merged_df['Citations Ratio'] = merged_df['Self-citations'] / merged_df['Citations']

# Encontrar el valor máximo de la nueva columna y el país correspondiente
max_citations_ratio_country = merged_df['Citations Ratio'].idxmax()
max_citations_ratio_value = merged_df['Citations Ratio'].max()

# Crear una tupla con el nombre del país y el valor máximo de la nueva columna
result = (max_citations_ratio_country, max_citations_ratio_value)

# Imprimir el resultado
print(result)


('Russian Federation', 0.36251678048211056)


### Pregunta 8

Crea una columna que estime la población utilizando `Energy Supply` y `Energy Supply per capita`.
¿Cuál es el tercer país más poblado según esta estimación?

*Esta función debería devolver el nombre del país*

In [325]:
def population():
    data = merged_df
    data['Population'] = data['Energy Supply'] / data['Energy Supply per Capita']
    data = data.sort_values(by = 'Population', ascending= False)
    thrid = data.iloc[2].name
    return thrid

poblacion = population()
print(poblacion)


Russian Federation


### Pregunta 9
Crea una columna que estime el número de `Citable documents` por persona.
¿Cuál es la correlación entre el número de `Citable documents` per cápita y el `Energy Supply per Capita`? Utiliza el método `.corr()`, (correlación de Pearson).

*Esta función debería devolver un solo número.*

*(Opcional: utiliza la función `plot9()` para visualizar la relación entre el suministro de energía per cápita y los `Citable documents` per cápita)*

In [333]:
# Crear la columna "Citable documents per Capita"
merged_df['Citable documents per Capita'] = merged_df['Citable documents'] / merged_df['Energy Supply per Capita']

# Calcular la correlación de Pearson
correlation = merged_df['Citable documents per Capita'].corr(merged_df['Energy Supply per Capita'])

# Imprimir la correlación
print(correlation)


-0.760842234092729


### Pregunta 10
Crea una nueva columna con un valor igual a 1 si el valor de % de energía renovable del país (`% Renewable`) es igual o superior de la mediana de todos los países en el top 15, o con un valor igual a 0 si el valor de % de energía renovable del país (`% Renewable`) está por debajo de la mediana.

*Esta función debería devolver una serie llamada `HighRenew` cuyo índice es el nombre del país ordenado ascendentemente según el ranking.*

In [337]:
# Calcular la mediana de la columna '% Renewable'
median_renewable = merged_df['% Renewable'].median()

# Crear la columna 'HighRenew' basada en la mediana
merged_df['HighRenew'] = merged_df['% Renewable'].apply(lambda x: 1 if x >= median_renewable else 0)

# Ordenar el DataFrame por el ranking
merged_df = merged_df.sort_values(by='Rank')

# Crear una serie llamada 'HighRenew' y establecer el índice como el nombre del país
HighRenew = merged_df['HighRenew']

# Imprimir la serie HighRenew
print(HighRenew)


Country
Russian Federation    0
Canada                1
Germany               1
India                 0
South Korea           0
Brazil                1
Name: HighRenew, dtype: int64


### Pregunta 11
Utiliza el siguiente diccionario para agrupar los países por continente, luego crea un DataFrame que muestre el tamaño de la muestra (la cantidad de países en cada contenedor de continente) y la suma, la media y la desviación estándar para la población estimada de cada país.

```python
continent_dict  = {'China':'Asia',
                   'United States':'North America',
                   'Japan':'Asia',
                   'United Kingdom':'Europe',
                   'Russian Federation':'Europe',
                   'Canada':'North America',
                   'Germany':'Europe',
                   'India':'Asia',
                   'France':'Europe',
                   'South Korea':'Asia',
                   'Italy':'Europe',
                   'Spain':'Europe',
                   'Iran':'Asia',
                   'Australia':'Australia',
                   'Brazil':'South America'}
```

*Esta función debería devolver un DataFrame con un índice llamado Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` y columnas `['size', 'sum', 'mean', 'std']`*

In [386]:
import pandas as pd
import numpy as np

def group_by_continent():
   
    data = merged_df
    
    # Diccionario de continentes
    continent_dict = {'China': 'Asia',
                      'United States': 'North America',
                      'Japan': 'Asia',
                      'United Kingdom': 'Europe',
                      'Russian Federation': 'Europe',
                      'Canada': 'North America',
                      'Germany': 'Europe',
                      'India': 'Asia',
                      'France': 'Europe',
                      'South Korea': 'Asia',
                      'Italy': 'Europe',
                      'Spain': 'Europe',
                      'Iran': 'Asia',
                      'Australia': 'Australia',
                      'Brazil': 'South America'}
    
   # Agregar la columna 'Continent' al DataFrame
    data['Continent'] = [continent_dict[country] for country in data.index]

    # Calcular la población estimada y agregarla al DataFrame
    data['Population'] = data['Energy Supply'] / data['Energy Supply per Capita']

    # Agrupar por continente y calcular las estadísticas
    grouped = data.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])
    
    return grouped

group_by_continent()

# Intenté, pero no pude añadir a Australia a la tabla, la misma solo tomo los datos de los 6 paises de merged_df del ejercicio 1

,size,sum,mean,std
Continent,,,,
Asia,2,1326536199.095022,663268099.547511,8.675672e+08
Europe,2,223869696.969697,111934848.484848,4.463987e+07
North America,1,35239864.864865,35239864.864865,NaN
South America,1,205915254.237288,205915254.237288,NaN
